## First import some necessary packages

In [1]:
import logging
import pathlib
import sys

import colorcet as cc
import dotenv
import geopandas as gpd
import holoviews as hv
import hvplot.pandas  # noqa: API import
import numpy as np
import pandas as pd
import panel as pn
import pooch
from bokeh.models import PanTool, WheelZoomTool

import coastal_dynamics as cd

# Activate Panel extension to make interactive visualizations
pn.extension()

In [2]:
# # Read questions from cloud storage
questions = cd.read_questions(
    "./6_cross_shore_transport.json"
    # "az://coastal-dynamics/questions/5_morphodynamics_upper_shoreface.json",
    # storage_options={"account_name": "coclico"},
)

# (Cross-shore) sediment transport
Welcome to the notebook of week 6! The main topic of this ntoebook is cross-shore sediment transport (chapter 7 of the book, with some extra attention for section 7.5 specifically). We will start with a small introduction, followed by a more detailed look into figure 7.21 from the book.

## Introduction
Remember from chapter 5 that the velocity $u$ close to the bed can be assumed to consists of a wave group-averaged component $\bar{u}$, a short-wave-averaged oscillatory component $u_{lo}$ and a short-wave component $u_{hi}$:
$$
u = \bar{u} + u_{lo} + u_{hi}
$$
We are interested in net sediment transport. We can use the third odd velocity moment as a proxy for this (which is a gross simplification, but a workable one):
$$
\left\langle u \left|u\right|^2\right\rangle = 3 \left\langle \bar{u} \left|u_{hi}\right|^2\right\rangle + \left\langle u_{hi} \left|u_{hi}\right|^2\right\rangle + 3 \left\langle u_{lo} \left|u_{hi}\right|^2\right\rangle + ...
$$
Each of these terms is fully explained in the book (so you should definitly read up on this), but in short, each of these terms refers to the following process:
* $3 \left\langle \bar{u} \left|u_{hi}\right|^2\right\rangle$ : transport related to the mean current (stirred up by short waves)
* $\left\langle u_{hi} \left|u_{hi}\right|^2\right\rangle$ : transport related to high-frequency waves (stirred up by short waves)
* $3 \left\langle u_{lo} \left|u_{hi}\right|^2\right\rangle$ : transport related to low-frequency waves (stirred up by short waves)

Each of the velocity moments can be directly measured in a flume, which is what Roelvink and Stive (1989) did. In quasi-steady sediment transport formulations, one can recognize the velocity moments. For instance, have a look at Eqs. 6.48a and 6.48b in this book. 

It is possible to derive an equillibrium profile mathematically by balancing onshore and offshore terms. Bowen (1980) does this analytically for the middle and lower shoreface by balancing onshore transport by short waves and and offshore transport by gravity.

For the present exercise, we are interested in Figure 7.21 and how it changes for different wave conditions and bathymetry. However, we are limited by the lack of direct observations. Tinker et al (2009) present a solution. They performed a large amount of measurements and fitted a shape function for both mean and oscillatory flow in the surf/shoaling zone, as well as onshore and offshore transport in the swash/surf zone. Though the paper is very interesting (and we definitely recommend checking it out), you are not required to know it for this exercise. You will hear more about it in the Coastal module as well, should you choose it!

However, before getting into the Tinker et al (2009) paper, let's first have a look at Figure 7.21 from the book. Use the velocity moments as a proxy for sediment transport. The total (net) transport consists of multiple gross contributions. 


![image](./6_fig720.png)

Using this figure, try to answer the questions below.

**Here we could start by asking a few questions about figure 7.21 (before loading the shape functions):** --> DONE
* What is the direction of the net transport? (**offshore**/onshore/depends on the distance from the shore)
* What is the direction of the transport by undertow (**offshore**/onshore/depends on the distance from the shore)
* What is the direction of the transport by skewness (offshore/**onshore**/depends on the distance from the shore)
* What is the direction of the transport by long waves (offshore/onshore/**depends on the distance from the shore**)
* Open question: Can you explain the direction of the transport due to long waves from the correlation between the bound long waves and the wave group? Hint: look at Figure 7.20 for this and read Section 5.8.2 in the textbook.
* For what conditions is figure 7.21 representative? (**storm conditions**, calm conditions)
* What morphological response can be expected? (**erosion higher in the profile**, erosion lower in the profile, **sedimentation lower in the profile**, sedimentation higher in the profile, **profile flattens**, profile steepens)

In [3]:
q1 = cd.QuestionFactory(questions["Q6-1"]).serve()
q2 = cd.QuestionFactory(questions["Q6-2"]).serve()
q3 = cd.QuestionFactory(questions["Q6-3"]).serve()
q4 = cd.QuestionFactory(questions["Q6-4"]).serve()
q5 = cd.QuestionFactory(questions["Q6-5"]).serve()
q6 = cd.QuestionFactory(questions["Q6-6"]).serve()

pn.Column(q1, q2, q3, q4, q5, q6)

Column
    [0] Column
        [0] StaticText(name='Q6-1', value='What is the d...)
        [1] RadioBoxGroup(name='Options', options=['offshore', 'onshore', ...], value='offshore')
        [2] Button(name='Submit')
        [3] StaticText()
    [1] Column
        [0] StaticText(name='Q6-2', value='What is the d...)
        [1] RadioBoxGroup(name='Options', options=['offshore', 'onshore', ...], value='offshore')
        [2] Button(name='Submit')
        [3] StaticText()
    [2] Column
        [0] StaticText(name='Q6-3', value='What is the d...)
        [1] RadioBoxGroup(name='Options', options=['offshore', 'onshore', ...], value='offshore')
        [2] Button(name='Submit')
        [3] StaticText()
    [3] Column
        [0] StaticText(name='Q6-4', value='What is the d...)
        [1] RadioBoxGroup(name='Options', options=['offshore', 'onshore', ...], value='offshore')
        [2] Button(name='Submit')
        [3] StaticText()
    [4] Column
        [0] StaticText(name='Q6-5', value='For what conditions i...)
        [1] RadioBoxGroup(name='Options', options=['storm conditions', ...], value='storm conditions')
        [2] Button(name='Submit')
        [3] StaticText()
    [5] Column
        [0] StaticText(name='Q6-6', value='What morphological r...)
        [1] CheckBoxGroup(name='Options', options=['erosion higher in the pr...])
        [2] Button(name='Submit')
        [3] StaticText()

## Define shape functions
Below the shape functions by Tinker et al (2009) are defined. They differentiate between transport in the surf/shoaling zone, and the swash/surf zone.
Transport in the surf/shoaling zone consists of mean and oscillatory transport, and transport in the swash/surf zone consists of an onshore and offshore component. 

**Note:** You are not expected to know or remember these equations, they are just here to help us with the exercise.

In [4]:
# Shape functions for the surf/shoal zone

def Tinker_mean(h, h_b):
    return (-120 * h_b**2) * (h / h_b)**4.3 * np.exp(-9.4 * h / h_b**0.75)

def Tinker_osci(h, h_b):
    return (2.75 * h_b**0.6) * (h / h_b)**3.5 * np.exp(-4.2 * h / h_b**1.05)

def Tinker_surf_shoal(h, h_b):
    return Tinker_mean(h, h_b) + Tinker_osci(h, h_b)

# Shape functions for the swash/surf zone

def Tinker_onsh(h, h_b):
    return 3.5 * h_b**1.9 * (h / h_b)**1.1 * np.exp(-31 * (h / h_b)**1.1)**(h_b**1.1)

def Tinker_offs(h, h_b):
    
    if h_b > 2.15:
        a_off = -3 * h_b + 4
    elif h_b <= 0.75:
        a_off = 0
    else:
        a_off = -1.25 * (h_b - 0.75)**2
        
    return a_off * h_b**1.1 * (h / h_b)**1.1 * np.exp(-5.7 * (h/h_b)**1.1)**(h_b**1.1)

def Tinker_swash_surf(h, h_b):
    return Tinker_onsh(h, h_b) + Tinker_offs(h, h_b)

# Total transport for swash/surf/shoal

def Tinker_total(h, h_b):
    return Tinker_surf_shoal(h, h_b) + Tinker_swash_surf(h, h_b)

Let's start with just looking at the surf/shoaling zone.
Let's try these functions for a for different types of bathymetry, assuming some breakpoint depth $h_b$! 
**maybe with text link to 3c notebook where based on H0 and T we calculate hb**

### Define bathymetry functions
The functions below define the bathymetry. You can select the bathymetry you want to look at using the selector menu in the plot that is to be generated.
**I did not get a selector menu, I also did not get a plot**
**Therefore I cannot really look at the notebook in depth. But here are some ideas for the story line of the remainder of the notebook:**
* First we have to explain some of the assumptions behind the Tinker paper: suspended sediment transport only, we are now using waves (short + long combined) and mean current as opposed to figure 7.21, bed profile and grain-size at the measurement location (could we have this profile as part of the bathymetries to be selected) and the range of hb values in the measurements.
* Explain the approach to the swash zone (which extends beyond the focus of 7.21)  
* Ask students to first select the profile for the measurement location and a hb to represent erosive conditions with net offshore transport. Ask questions about: -range of hb values for which erosive conditions, -for a particular hb in this range: width of surf zone (in m), direction of transport due waves and due to undertow, zones in which onshore and offshore transport are significant (as a function of cross-shore distance and/or h/hb) and direction of net transport (also as a function of cross-shore distance and/or h/hb)
* Let them compare the results to Figure 7.21.
* Now let them determine the hb range for accretive conditions. Choose a certain hb and answer again the questions about width of surf zone (in m), direction of transport due waves and due to undertow, zones in which onshore and offshore transport are significant (as a function of cross-shore distance and/or h/hb) and direction of net transport (also as a function of cross-shore distance and/or h/hb)
* Now ask reflective questions about the difference between the accretive conditions and erosive conditions
* Now discuss validity for different situations and let them choose a typical Dutch profile. Now let them again find the range of hb for which there are typical erosive and accretive conditions.
* Let them compare the UK situation to the Dutch situation
* Now introduce the way of plotting in terms of h/hb in the paper and show these plots. Maybe somne questions to interpret this.
* To be determined: which role do we have for the various bathymetries

In [9]:
# Bathymetries here!
uk_bath = pd.read_csv("6_uk_bath.csv", sep='; ', decimal=',', names=['x', 'y'])
nl_bath = pd.read_csv("6_nl_bath.csv", sep='; ', decimal=',', names=['x', 'y'])

line1 = uk_bath.hvplot(x='x', y='y', label='UK')
line2 = nl_bath.hvplot(x='x', y='y', label='NL')

line1 * line2

# display(uk_bath)

C:\Users\khdeb\AppData\Local\Temp\ipykernel_61024\2933899311.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  uk_bath = pd.read_csv("6_uk_bath.csv", sep='; ', decimal=',', names=['x', 'y'])
C:\Users\khdeb\AppData\Local\Temp\ipykernel_61024\2933899311.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  nl_bath = pd.read_csv("6_nl_bath.csv", sep='; ', decimal=',', names=['x', 'y'])
C:\Users\khdeb\mambaforge\envs\coastal\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated 

:Overlay
   .Curve.UK :Curve   [x]   (y)
   .Curve.NL :Curve   [x]   (y)

In [ ]:
MSL slider
hb slider

bath_selector


### Define plot function

**Note**: Although you don't have to understand the plot method, we include it here so you can see how these interactive plots are made! !

In [6]:
def show_transport(x, plot_where="pop-out", n_values=100, hb_start=0.8, max_depth=3, margin=0.05):
    """
    change value of 'plot_where' to:
    'inline' if you would like the plot to show in the notebook
    'pop-out' if you would like the plot to show in a new tab (i.e. seperate window)
    """
    
    # hb_slider = pn.widgets.FloatSlider(name="Breaking depth", start=h[1], end=h[-2], value=h[int(0.8*len(h))], step=h[2] - h[1])
    hb_slider = pn.widgets.FloatSlider(name="Breaking depth", start=0.1 * max_depth, end=0.9 * max_depth, step=0.05, value=hb_start)

    bath_dropdown =  pn.widgets.Select(name="Bathymetry select", options=["Linear", "Double linear", "Sigmoid", "Bar", "Double bar"], value="Linear")

    @pn.depends(hb_slider.param.value, bath_dropdown.param.value)
    def plot(h_b, bath_type):
        h = generate_bathymetry(bath_type, x, max_depth, margin=margin)

        x_b = x[np.argmin(np.abs(h - h_b))]
        
        # print(x[np.argwhere(h==h_b)])
        # x_b = x[np.argwhere(h==h_b)][0,0]
        
        bath_plot = hv.Curve((x, -h), label='Bathymetry').opts(xlabel='x [m]', ylabel='z [m]') * \
                    hv.HLine(0, label='water level').opts(line_dash='dashed') * \
                    hv.VLine(x_b, label='location of breaking').opts(line_dash='dashed')

        bath_plot = bath_plot.opts(show_legend=True)
        
        mean_transport = Tinker_mean(h, h_b)
        osci_transport = Tinker_osci(h, h_b)
        total_transport = Tinker_total(h, h_b)
        
        transport_curves = hv.Curve((x, mean_transport), label='mean transport') * \
                           hv.Curve((x, osci_transport), label='oscillatory transport') * \
                           hv.Curve((x, total_transport), label='total transport')
        
        transport_plot = transport_curves.opts(xlabel='', ylabel='')
        
        p = (bath_plot.opts(
            height=250, width=800, show_grid=True) + \
         transport_plot.opts(
             height=250, width=1200, show_grid=True)).opts(shared_axes=False).cols(1)

        return p

    app = pn.Column(bath_dropdown, hb_slider, plot)
    
    if plot_where == "inline":
        return app
    elif plot_where == "pop-out":
        app.show()
    else:
        print("please use either inline or pop-out for the plot_where variable")

### Define cross-shore domain
Finally, we define our cross-shore domain.

In [7]:
# Set x-domain
x = np.linspace(0, 50, 1000)

### Now plot the results

Execute the cell below to generate the plot by using the function we defined above. Please note that altering the slider positions or selecting different options from the dropdown menus may trigger a warning; it can safely be ignored, and possibly silenced by the adjusting the logging warning level. 

In [8]:
logging.getLogger().setLevel(logging.ERROR)

show_transport(x, plot_where="inline", max_depth=3)


# LOOK AT DIFFERENT UNIQUE VALUES IN H ARRAY, AND USE THOSE AS OPTIONS FOR H_B SLIDER!!!

NameError: name 'generate_bathymetry' is not defined

TODOS:
* FIX AXIS TOGETHER
* MAKE FIGURE NON-INTERACTABLE
* MAKE FIGURES MOVE TOGETHER
* PLOT OTHER TRANSPORT COMPONENTS
* ADD TINKER PLOT WHERE EVERYTHING IS STACKED TOGETHER
* PRESENT IN TWO FIGURES TO HELP STUDENTS COMPARE HIGH WAVES VS LOW WAVES

* MAKE QUESTIONS (idea: focus on when these shape functions might be valid or not, i.e. different situations)
* ADD MARKDOWNS


# Questions about:
- unit of Q
- interpretation of different scenarios